# Julia

- High level language for numerical computing
- https://julialang.org currently at v1.0.1
- Matlab-like syntax with C-like performance
- Code is compiled "on the fly"
- Excellent meta-programming capabilities

# A First Example

In [1]:
# Define the pdf of a standard normal distribution (could also be defined as a one-liner)
function standardNormalPdf(x::Float64)
    (1/sqrt(2*pi))*exp(-0.5*x^2)
end

standardNormalPdf (generic function with 1 method)

In [2]:
standardNormalPdf(0.0)

0.3989422804014327

# Closures

In [3]:
# A closure is a function that returns a function
# Closures can be used to shield local variables from the main scope
# We may think of these local variables as hidden states

# Define a closure with a hidden (locally scoped) state z
function initializeWorld()
    z_t_min = 0.0 # Initial state
    function execute(a_t::Float64) # Define how an action affects the hidden state
        z_t = z_t_min + a_t # State transition model
        x_t = z_t + randn() # Observation model
        
        z_t_min = z_t # Prepare for next step
    end
    
    x_t = 0.0
    observe() = x_t # Observe function provides access to the local variable x_t

    return (execute, observe) # Expose the functions of the closure
end

initializeWorld (generic function with 1 method)

In [4]:
(execute, observe) = initializeWorld() # Let there be a world with which we can interact

(execute, observe)

In [5]:
T = 100 # Time horizon
y = Vector{Float64}(undef, T) # Preset observations vector
a = sin.(2*pi*(1:T)./50) # Predefine some arbitrary policy
for t = 1:T # Step through an interaction protocol
    execute(a[t]) # Execution affects hidden states in the world
    y[t] = observe() # Emit an observation
end

In [6]:
using PyPlot

# Inspect the observations
plot(1:T, y, color="black", lw=2)
grid("on")
xlabel("Time t")
ylabel("Observations y")
;

# Metaprogramming

In [7]:
# Metaprogramming allows us to change a program before we run it

using Distributions # Load the external Distributions.jl package

x = rand(Normal(2,3), 10) # Standard syntax for drawing 10 samples from a normal distribution; a bit ugly

# We desire a domain-specific syntax that allows us to type:
# @sample x ~ Normal(2,3)

10-element Array{Float64,1}:
  4.428260672250293  
 -0.20371773806022153
 -2.007800832188135  
  7.211313372382085  
  4.56263824137624   
 -4.239021198616685  
 -0.9748844124793306 
  5.185576029633877  
  0.01656507960852882
 -1.0757286150530834 

In [8]:
expr = Meta.parse("x ~ Normal(2,3)") # Inspect our desired expression
dump(expr)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol ~
    2: Symbol x
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol Normal
        2: Int64 2
        3: Int64 3


In [9]:
# Declare our domain-specific syntax through a macro definition
macro sample(expr::Expr)
    (expr.args[1] == :~) || error("Invalid call to @sample") # Check for valid expression
    var  = expr.args[2] # Extract output variable
    dist = expr.args[3] # Extract distribution expression
    str  = "$var = rand($dist, 10)" # Generate code for evaluation
    eval(Meta.parse(str)) # Parse and evaluate generated code
end

@sample (macro with 1 method)

In [10]:
@sample x ~ Normal(2,3) # Check out our new syntax

10-element Array{Float64,1}:
 -1.042175018267618 
  3.281168191349047 
 -0.3904143709490202
  1.3039711964668115
  3.609208644944662 
  2.1816498057808116
 -2.866473238049961 
  0.985296110421688 
 -4.110720615431154 
  2.6726464172373277

In [11]:
@sample y ~ Gamma(2,3) # @sample automatically works with all distribution types defined in Distributions.jl

10-element Array{Float64,1}:
  1.3896685415459975
  8.640423121824611 
  2.3705726794737503
  5.458223635791514 
  4.189420357678381 
 14.32664393769889  
 10.443844794349694 
  1.4601501571207605
  2.4144186109441184
  3.3912542340338194